# Web scraping on Hubble Website

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
site = 'https://esahubble.org/images/'

response = requests.get(site)

soup = BeautifulSoup(response.text, 'html.parser')
img_tags = soup.find_all('img')

urls = [img["src"] for img in img_tags]

In [3]:
images = soup.find_all("script")[0].text.split("url")[1:]
urls = [site+url.split("'")[1].split("/images/")[1] for url in images if url.__contains__("jpg")]
urls

['https://esahubble.org/images/heic2017a/',
 'https://esahubble.org/images/heic2007a/',
 'https://esahubble.org/images/heic1509a/',
 'https://esahubble.org/images/heic1501a/',
 'https://esahubble.org/images/potw2050a/',
 'https://esahubble.org/images/heic2018b/',
 'https://esahubble.org/images/heic0715a/',
 'https://esahubble.org/images/heic1608a/',
 'https://esahubble.org/images/potw1345a/',
 'https://esahubble.org/images/heic1307a/',
 'https://esahubble.org/images/heic0817a/',
 'https://esahubble.org/images/heic0406a/',
 'https://esahubble.org/images/potw2049a/',
 'https://esahubble.org/images/opo0328a/',
 'https://esahubble.org/images/heic1917a/',
 'https://esahubble.org/images/heic0702a/',
 'https://esahubble.org/images/heic0515a/',
 'https://esahubble.org/images/heic1808a/',
 'https://esahubble.org/images/potw1811a/',
 'https://esahubble.org/images/heic0910h/',
 'https://esahubble.org/images/heic0601a/',
 'https://esahubble.org/images/heic0514a/',
 'https://esahubble.org/images/he

In [4]:
img_urls = []
img_categ = []
for url in urls[:10]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    img_tags = soup.find_all('img')[2]["src"]
    categorie = soup.find("div", {"class": "object-info"}).find_all("a")[-1].text
    img_urls.append(img_tags)
    img_categ.append(categorie)
img_urls, img_categ

(['https://cdn.spacetelescope.org/archives/images/thumb700x/heic2017a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/heic2007a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/heic1509a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/heic1501a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/potw2050a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/heic2018b.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/heic0715a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/heic1608a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/potw1345a.jpg',
  'https://cdn.spacetelescope.org/archives/images/thumb700x/heic1307a.jpg'],
 ['Solar System',
  'Nebulae',
  'Star Clusters',
  'Nebulae',
  'Cosmology',
  'Galaxies',
  'Star Clusters',
  'Nebulae',
  'Galaxies',
  'Nebulae'])

In [11]:
data = pd.DataFrame(columns=["file", "target"])
for idx in range(len(img_urls)):
    filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', img_urls[idx])
    if not filename:
         print("Regex didn't match with the url: {}".format(url))
         continue
    with open("../data/"+filename.group(1), 'wb') as f:
        response = requests.get(img_urls[idx])
        f.write(response.content)
        curr = pd.DataFrame([[filename.group(1), img_categ[idx]]], columns=["file", "target"])
        data = pd.concat([curr, data])

data.to_csv("../data/labelled_pictures.csv", sep=";", index=False)

In [12]:
data

,file,target
0,heic1307a.jpg,Nebulae
0,potw1345a.jpg,Galaxies
0,heic1608a.jpg,Nebulae
0,heic0715a.jpg,Star Clusters
0,heic2018b.jpg,Galaxies
0,potw2050a.jpg,Cosmology
0,heic1501a.jpg,Nebulae
0,heic1509a.jpg,Star Clusters
0,heic2007a.jpg,Nebulae
0,heic2017a.jpg,Solar System
